<a href="https://colab.research.google.com/github/cs23m066/Assignment_2/blob/main/dlassignment_2_part_b_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback
import socket
socket.setdefaulttimeout(30)
wandb.login()
wandb.init(project ='DL_assignment_2_B')

2024-04-06 08:46:44.698280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-06 08:46:44.698399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-06 08:46:44.872918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23m066 (dlassignment). Use `wandb login --relogin` to force relogin


In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip

--2024-04-06 08:49:12--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.207, 142.250.107.207, 74.125.20.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: 'nature_12K.zip'

nature_12K.zip      100%[===================>]   3.55G   250MB/s    in 14s     

2024-04-06 08:49:27 (253 MB/s) - 'nature_12K.zip' saved [3816687935/3816687935]



In [ ]:
!rm nature_12K.zip

rm: cannot remove 'nature_12K.zip': No such file or directory


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])

train_dataset = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform)
train_dataset,val_dataset = torch.utils.data.random_split(train_dataset,[8000,1999])

transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by a maximum of 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(224),  # Randomly crop and resize the image to 224x224
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5,),(0.5,))  # Normalize the image
])

train_dataset2 = datasets.ImageFolder(root='inaturalist_12K/train',transform=transform2)
train_dataset_aug,val_dataset_aug = torch.utils.data.random_split(train_dataset2,[8000,1999])

def dataFun(aug,batchSize):
    if(aug == 'no'):
        train_loader = torch.utils.data.DataLoader(train_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        val_loader = torch.utils.data.DataLoader(val_dataset,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)
        return train_loader,val_loader
    else:
        train_loader_aug = torch.utils.data.DataLoader(train_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        val_loader_aug = torch.utils.data.DataLoader(val_dataset_aug,batch_size =batchSize,shuffle = True,num_workers=4,pin_memory=True)
        return train_loader_aug,val_loader_aug

In [ ]:
def RESNET50(NUM_OF_CLASSES):
    model = models.resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)

    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc.parameters():
        param.requires_grad = True

    return model

In [ ]:
def RESNET50_1(k,NUM_OF_CLASSES):
    model = models.resnet50(pretrained=True)

    params = list(model.parameters())
    for param in params[:k]:
        param.requires_grad = False

    num_ftrs = model.fc.in_features

    model.fc = torch.nn.Linear(num_ftrs, NUM_OF_CLASSES)

    return model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_fun(batchSize,num_epochs,learning_rate,aug,strategy,NUM_OF_CLASSES):
    aug = 'no'

    train_loader,val_loader = dataFun(aug,batchSize)
    if(strategy == 0):
        model = RESNET50(NUM_OF_CLASSES).to(device)
    else:
        model = RESNET50_1(10,NUM_OF_CLASSES).to(device)


    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)
            # forward
            scores = model(data)
            loss = criterion(scores,targets)
            # backward
            optimizer.zero_grad()
            loss.backward()
            # gradient descent or adam step
            optimizer.step() #updates the parameters
        train_accuracy,train_loss = check_accuracy(train_loader, model,criterion,batchSize)
        validation_accuracy,validation_loss = check_accuracy(val_loader, model,criterion,batchSize)
        print(f"train_accuracy:{train_accuracy:.4f},train_loss:{train_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})

    #wandb.log({'train_accuracy':train_accuracy})


In [ ]:
def check_accuracy(loader,model,criterion,batchSize):
    num_correct = 0
    num_loss = 0
    total = 0
    num_samples = 0
    total_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batchSize
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples)*100 , total_loss

In [ ]:
batchSize = 32
num_epochs = 30
learning_rate = 1e-4
aug = 'no'
NUM_OF_CLASSES = 10
strategy = 1
train_fun(batchSize,num_epochs,learning_rate,aug,strategy,NUM_OF_CLASSES)

100%|██████████| 250/250 [01:12<00:00,  3.45it/s]


train_accuracy:89.2500,train_loss:2776.6125
validation_accuracy:75.3877,validation_loss:1508.3701


 37%|███▋      | 93/250 [00:27<00:46,  3.41it/s]


KeyboardInterrupt: 

In [ ]:
def main_fun():
    wandb.init(project ='DL_assignment_2_B')
    params = wandb.config
    with wandb.init(project = 'DL_assignment_2_B', name='strategy'+str(params.strategy)+'batchSize'+str(params.batchSize)+'num_epochs'+str(params.num_epochs)+'learning_rate'+str(params.learning_rate)) as run:
        train_fun(params.batchSize,params.num_epochs,params.learning_rate,params.aug,params.strategy,10)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'Shriram',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'batchSize':{'values':[32,64]},
        'num_epochs':{'values':[10,20]},
        'learning_rate':{'values' : [1e-3,1e-4]},
        'aug' :{'values':['no','yes']},
        'strategy' :{'values' : [0]}
    }
}
sweepId = wandb.sweep(sweep_params,project = 'DL_assignment_2_B')
wandb.agent(sweepId,function =main_fun,count = 4)
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6f6392e200>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f6e24be47c0, raw_cell="def main_fun():
    wandb.init(project ='DL_assign.." store_history=True silent=False shell_futures=True cell_id=572732ff-441e-481c-900b-1907bc34a2fd>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Create sweep with ID: 1evbqxjz
Sweep URL: https://wandb.ai/dlassignment/DL_assignment_2_B/sweeps/1evbqxjz


wandb: Agent Starting Run: ycmog8ja with config:
wandb: 	aug: no
wandb: 	batchSize: 64
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: 	strategy: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

100%|██████████| 125/125 [00:46<00:00,  2.70it/s]


train_accuracy:72.2125,train_loss:8533.6561
validation_accuracy:71.8859,validation_loss:2223.1588


100%|██████████| 125/125 [00:45<00:00,  2.75it/s]


train_accuracy:73.3500,train_loss:7775.5311
validation_accuracy:72.3362,validation_loss:2059.1777


 34%|███▎      | 42/125 [00:15<00:27,  2.99it/s]